In [1]:
import glob
import numpy as np

u = "unclassified"
data = {}
totals = {}

ref_tool = "reference"
max_len = len("V300024282_L02_104")


In [2]:
# fastdna
data["fastdna"] = {}
totals["fastdna"] = {}

for fname in glob.glob("*.fastdna.classif"):
    print(fname)
    proc_name = fname.split(".")[0]
    data["fastdna"][proc_name] = {}
    totals["fastdna"][proc_name] = 0
    
    with open(fname) as f:
        for l in f.read().split("\n"):
            totals["fastdna"][proc_name] += 1
            
            t = l.strip()
            if t == "":
                t = u
                
            if t not in data["fastdna"][proc_name]:
                data["fastdna"][proc_name][t] = 0
            data["fastdna"][proc_name][t] += 1
    
# kraken2
data["kraken2"] = {}
totals["kraken2"] = {}

for fname in glob.glob("*.kraken2_paired.classif"):
    print(fname)
    proc_name = fname.split(".")[0]
    data["kraken2"][proc_name] = {}
    totals["kraken2"][proc_name] = 0
    
    with open(fname) as f:
        for l in f.read().split("\n"):
            pl = l.split("\t")
            
            if len(pl) > 2:
                totals["kraken2"][proc_name] += 1

                t = pl[2]
                if pl[0] == "U":
                    t = u

                if t not in data["kraken2"][proc_name]:
                    data["kraken2"][proc_name][t] = 0
                data["kraken2"][proc_name][t] += 1
                
# metavw
data["metavw"] = {}
totals["metavw"] = {}

for fname in glob.glob("*.preds.vw"):
    print(fname)
    proc_name = fname.split(".")[0]
    data["metavw"][proc_name] = {}
    totals["metavw"][proc_name] = 0
    
    with open(fname) as f:
        for l in f.read().split("\n"):
            t = l.strip()
                
            if t != "":
                totals["metavw"][proc_name] += 1

                if t not in data["metavw"][proc_name]:
                    data["metavw"][proc_name][t] = 0
                data["metavw"][proc_name][t] += 1  

# deepmicrobes
data["deepmicrobes"] = {}
totals["deepmicrobes"] = {}

for fname in glob.glob("*.result.txt"):
    print(fname)
    proc_name = fname.replace(".result.txt", "")
    data["deepmicrobes"][proc_name] = {}
    totals["deepmicrobes"][proc_name] = 0
    
    with open(fname) as f:
        for l in f.read().split("\n"):
            t = l.strip()
                
            if t != "":
                totals["deepmicrobes"][proc_name] += 1

                t_split = t.split("\t")[0]
                if pl[0] == "U":
                    t = u
                if t_split not in data["deepmicrobes"][proc_name]:
                    data["deepmicrobes"][proc_name][t_split] = 0
                data["deepmicrobes"][proc_name][t_split] += 1  
    

V300024282_L02_1.fq.gz.fastdna.classif
control_2.fq.gz.fastdna.classif
V300024282_L02_104.fq.gz.fastdna.classif
V300024282_L02_97.fq.gz.fastdna.classif
control_1.fq.gz.fastdna.classif
V300024282_L02_104.fq.gz.kraken2_paired.classif
V300024282_L02_97.fq.gz.kraken2_paired.classif
control_1.fq.gz.kraken2_paired.classif
control_2.fq.gz.kraken2_paired.classif
V300024282_L02_1.fq.gz.kraken2_paired.classif
control_2.fq.gz.preds.vw
V300024282_L02_1.fq.gz.preds.vw
control_1.fq.gz.preds.vw
V300024282_L02_97.fq.gz.preds.vw
V300024282_L02_104.fq.gz.preds.vw
V300024282_L02_1.result.txt
control_2.result.txt
V300024282_L02_104.result.txt
V300024282_L02_97.result.txt
control_1.result.txt


In [3]:
totals["kraken2"]


{'V300024282_L02_104': 22358511,
 'V300024282_L02_97': 28987851,
 'control_1': 38581675,
 'control_2': 52537916,
 'V300024282_L02_1': 28990760}

In [3]:
data[ref_tool] = dict(data["kraken2"])
totals[ref_tool] = dict(totals["kraken2"])
    
# taxids gathered up in accordance to the theoretical distribution of organisms available in https://files.zymoresearch.com/protocols/_d6300_zymobiomics_microbial_community_standard.pdf
data[ref_tool]["control_1"] = {
    "287": totals[ref_tool]["control_1"]*0.12,
    "562": totals[ref_tool]["control_1"]*0.12,
    "28901": totals[ref_tool]["control_1"]*0.12,
    "1613": totals[ref_tool]["control_1"]*0.12,
    "1351": totals[ref_tool]["control_1"]*0.12,
    "1280": totals[ref_tool]["control_1"]*0.12,
    "1639": totals[ref_tool]["control_1"]*0.12,
    "1423": totals[ref_tool]["control_1"]*0.12,
    "4932": totals[ref_tool]["control_1"]*0.02,
    "235443": totals[ref_tool]["control_1"]*0.02
}

data["reference"]["control_2"] = {
    "287": totals[ref_tool]["control_2"]*0.12,
    "562": totals[ref_tool]["control_2"]*0.12,
    "28901": totals[ref_tool]["control_2"]*0.12,
    "1613": totals[ref_tool]["control_2"]*0.12,
    "1351": totals[ref_tool]["control_2"]*0.12,
    "1280": totals[ref_tool]["control_2"]*0.12,
    "1639": totals[ref_tool]["control_2"]*0.12,
    "1423": totals[ref_tool]["control_2"]*0.12,
    "4932": totals[ref_tool]["control_2"]*0.02,
    "235443": totals[ref_tool]["control_2"]*0.02
}


In [4]:
totals_scaled = dict(totals)

# there's some skipping weirdness w fastdna
for fname in totals_scaled["fastdna"]:
    if u not in data["fastdna"][fname]:
        data["fastdna"][fname][u] = 0
    # this should NOT print any negatives
    print(totals_scaled["metavw"][fname]-totals_scaled["fastdna"][fname])
    data["fastdna"][fname][u] += totals_scaled["metavw"][fname]-totals_scaled["fastdna"][fname]
    totals_scaled["fastdna"][fname] = totals_scaled["metavw"][fname]

5118876
8613002
3091399
7039032
5741165


In [5]:
nodes = {}

with open('nodes.dmp') as f:
    for x in f.read().split("\n"):
        xs = x.split("\t|\t")
        
        if len(xs) > 1:
            nodes[xs[0]] = {
                "parent": xs[1],
                "rank": xs[2]
            }

def translate_taxid(taxid, target_rank, nodes, u):
    if taxid not in nodes or nodes[taxid]["parent"] == taxid:
        return u
    else:
        if nodes[taxid]["rank"] == target_rank:
            return taxid 
        else:
            return translate_taxid(nodes[taxid]["parent"], target_rank, nodes, u) 

target = "phylum"
phylum_data = {}

for tool, files in data.items():
    phylum_data[tool] = {}
    
    for fname, classifs in files.items():
        phylum_data[tool][fname] = {}
        
        for c, v in classifs.items():
            new_c = translate_taxid(c, target, nodes, u)
            
            if new_c not in phylum_data[tool][fname]:
                phylum_data[tool][fname][new_c] = 0
            phylum_data[tool][fname][new_c] += v
            
target = "genus"
genus_data = {}

for tool, files in data.items():
    genus_data[tool] = {}
    
    for fname, classifs in files.items():
        genus_data[tool][fname] = {}
        
        for c, v in classifs.items():
            new_c = translate_taxid(c, target, nodes, u)
            
            if new_c not in genus_data[tool][fname]:
                genus_data[tool][fname][new_c] = 0
            genus_data[tool][fname][new_c] += v  
            

In [6]:
totals


{'fastdna': {'V300024282_L02_1': 7842209,
  'control_2': 13407150,
  'V300024282_L02_104': 4747570,
  'V300024282_L02_97': 10795445,
  'control_1': 8901101},
 'kraken2': {'V300024282_L02_104': 22358511,
  'V300024282_L02_97': 28987851,
  'control_1': 38581675,
  'control_2': 52537916,
  'V300024282_L02_1': 28990760},
 'metavw': {'control_2': 13407150,
  'V300024282_L02_1': 7842209,
  'control_1': 8901101,
  'V300024282_L02_97': 10795445,
  'V300024282_L02_104': 4747570},
 'deepmicrobes': {'V300024282_L02_1': 28990760,
  'control_2': 52537916,
  'V300024282_L02_104': 22358511,
  'V300024282_L02_97': 28987851,
  'control_1': 38581675},
 'reference': {'V300024282_L02_104': 22358511,
  'V300024282_L02_97': 28987851,
  'control_1': 38581675,
  'control_2': 52537916,
  'V300024282_L02_1': 28990760}}

In [7]:
# coverage with kraken2 being trusted to remember the total number of reads
for fname in totals[ref_tool]:
    print(fname+" ".join(["" for x in range(max_len-len(fname))])+"\t|\tbaseline\t|\tphylum    \t|\tgenus")
    
    for tool in totals:
        if fname in data[tool]:
            ref_total = totals_scaled[tool][fname]

            classified = 0
            for k, v in data[tool][fname].items():
                if k != u:
                    classified += v

            classified_phylum = 0
            for k, v in phylum_data[tool][fname].items():
                if k != u:
                    classified_phylum += v

            classified_genus = 0
            for k, v in genus_data[tool][fname].items():
                if k != u:
                    classified_genus += v

            print(tool+":"+" ".join(["" for x in range(max_len-len(tool))])+"\t|\t{:.6f}\t|\t{:.6f}\t|\t{:.6f}".format(
                classified/ref_total, 
                classified_phylum/ref_total,
                classified_genus/ref_total
            ))


V300024282_L02_104	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.084772	|	0.084772	|	0.084358
kraken2:          	|	0.052410	|	0.051654	|	0.025456
metavw:           	|	1.000000	|	1.000000	|	0.924515
deepmicrobes:     	|	1.000000	|	0.679685	|	0.590301
reference:        	|	0.052410	|	0.051654	|	0.025456
V300024282_L02_97	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.084387	|	0.084387	|	0.083974
kraken2:          	|	0.069359	|	0.068415	|	0.033696
metavw:           	|	1.000000	|	1.000000	|	0.905263
deepmicrobes:     	|	1.000000	|	0.674508	|	0.608315
reference:        	|	0.069359	|	0.068415	|	0.033696
control_1        	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.086018	|	0.086018	|	0.085608
kraken2:          	|	0.849497	|	0.849210	|	0.842159
metavw:           	|	1.000000	|	1.000000	|	0.998415
deepmicrobes:     	|	1.000000	|	0.742349	|	0.703307
reference:        	|	1.000000	|	1.000000	|	1.000000
control_2        	|	baseline	|	phylum    	|	genus
fastdna:          	

In [8]:
for fname in totals[ref_tool]:
    print(fname+" ".join(["" for x in range(max_len-len(fname))])+"\t|\tbaseline\t|\tphylum    \t|\tgenus")
    
    for tool in totals:
        if fname in data[tool]:
            ref_total = totals_scaled[ref_tool][fname]

            accurate = 0
            for c, v in data[tool][fname].items():
                if c in data[ref_tool][fname]:
                    accurate += min(data[ref_tool][fname][c]/ref_total, v/totals_scaled[tool][fname])

            accurate_phylum = 0
            for c, v in phylum_data[tool][fname].items():
                if c in phylum_data[ref_tool][fname]:
                    accurate_phylum += min(phylum_data[ref_tool][fname][c]/ref_total, v/totals_scaled[tool][fname])

            accurate_genus = 0
            for c, v in genus_data[tool][fname].items():
                if c in genus_data[ref_tool][fname]:
                    accurate_genus += min(genus_data[ref_tool][fname][c]/ref_total, v/totals_scaled[tool][fname])

            print(tool+":"+" ".join(["" for x in range(max_len-len(tool))])+"\t|\t{:.6f}\t|\t{:.6f}\t|\t{:.6f}".format(
                accurate,#/ref_total, 
                accurate_phylum,#/ref_total,
                accurate_genus,#/ref_total
            ))


V300024282_L02_104	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.918670	|	0.952551	|	0.926846
kraken2:          	|	1.000000	|	1.000000	|	1.000000
metavw:           	|	0.011609	|	0.051654	|	0.100941
deepmicrobes:     	|	0.000469	|	0.334992	|	0.411494
reference:        	|	1.000000	|	1.000000	|	1.000000
V300024282_L02_97	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.920867	|	0.953436	|	0.932710
kraken2:          	|	1.000000	|	1.000000	|	1.000000
metavw:           	|	0.013381	|	0.068415	|	0.128433
deepmicrobes:     	|	0.000599	|	0.342215	|	0.393848
reference:        	|	1.000000	|	1.000000	|	1.000000
control_1        	|	baseline	|	phylum    	|	genus
fastdna:          	|	0.010109	|	0.063727	|	0.023855
kraken2:          	|	0.334831	|	0.849112	|	0.666657
metavw:           	|	0.666870	|	0.925995	|	0.677907
deepmicrobes:     	|	0.000000	|	0.360000	|	0.061839
reference:        	|	1.000000	|	1.000000	|	1.000000
control_2        	|	baseline	|	phylum    	|	genus
fastdna:          	